# SQL
___

SQL (por sus siglas en inglés) es un lenguaje estructurado de consultas entre datos y algo llamado _"Relational Database"_. Este último termino se refiere a una colección especifica de tablas.

Cada tabla representa una entidad y está compuesta por atributos (columnas) y elementos (renglones). Los atributos de la siguiente tabla son `id`, `name`, `age` y `nationality`. Cada elemento (renglón) representa los datos personales de cada una de las siguientes personas.  

|id|name|age|nationality|
|--|----|---|-----------|
|1|Jessica|22|Ireland|
|2|Gabriel|48|France|
|3|Laura|36|USA|

### Psycopg

Psycopg es el adaptador más popular de PostgreSQL para Python, con el cual podemos convertir la interfaz de jupyter en PostgreSQL, realizando cualquier actividad como: creación, actualización y gestión de bases de datos.

Esta herramienta nos ayudará como canal para cmunicarnos desde Python con nuestras bases de datos y con ella poder realizar cualquier tipo de consulta.

Para ello necesitamos cargar la libreriía `psycopg2`, crear el objeto `conn = psycopg2.connect("dbname=test user=postgres")` para poder conectarnos con la base de datos de nuestra elección, luego el objeto `cur = conn.cursor()` que abre la comunicación con la base de datos. A partir de aquí lo siguiente es construir las consultas mediante el objeto `cur.execute`. Por ejemplo: `cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")` o `cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)", (100, "abc'def"))`.

Si queremos capturar la respuesta del _Query_ que hemos realizado a una base de datos y visualizarla como _output_ en jupyter, utilizamos el objeto `cur.fetchone()`. Por ejemplo: necesitamos obtener el resultado de la consulta `cur.execute("SELECT * FROM test;")`. Entonces bastará con agregar, al final del _statement_ anterior el objeto `cur.fetchone()`:

```sql
cur.execute("SELECT * FROM test;")
cur.fetchone()
(1, 100, "abc'def")
```
Para poder realizar los cambios, desde Python a una tabla en una base de datos de PostgreSQL, utilizamos el objeto `conn.commit()`. Y para finalizar la comunicación con la base de datos los objetos `cur.close()` y `conn.close()`.



### SELECT (_selección de columnas_).

Nos concentraremos en la consulta de la información a una tablas. Para ello es necesario contruir la sintaxis que realizará esta tarea, también conocido como _"queriying"_. 

Un _"Query"_ es la construcción de la sintaxis (_statement_), en _**SQL**_, para poder obtener una respuesta con los datos de una tabla o para generar nueva información o para actualizar una tabla preexistente. En _**SQL**_ podemos seleccionar una tabla usando las _keys words_ **SELECT** y **FROM** en mayúsculas como buena práctica. Por ejemplo, la siguiente sintaxis (_statement_) selecciona la columna `name` de la tabla `people`:

```sql
SELECT name FROM people;
```

Si utilizamos esquemas el _statement_ cambia: `SELECT tabla.columna FROM esquema.tabla;`. 

Para poder realizar la elección de de múltiples columnas bastará con especificar el nombre de cada una de ellas. Por ejemplo;

```sql
SELECT name, birthdate FROM people;
```

O en su lugar seleccionar todas los atributos de la tabla utilizando `*`:

```sql
SELECT * FROM people;
```

Para limitar el número de elementos (_renglones_) de una consulta utilizamos la _key word_ `LIMIT`:

```sql
SELECT * FROM people LIMIT 10;
```

Si el resultado de nuestra consulta presume duplicados, `DISTINCT` selecciona solo los records **únicos** en un atributo:

```sql
SELECT DISTINCT rol FROM roles;
```

Si quicieramos realizar el conteo de valores existentes en un atributo, `COUNT` retorna el número de elementos:

```sql
SELECT COUNT(*) FROM people;
```

Mientras tanto, si necesitamos realizar un conteo de valores únicos _**non-missing values**_, a diferencia de utilizar `*`, `COUNT(DISTINCT atributo)` nos ayuda a realizar esta tarea;

```sql
SELECT COUNT(DISTINCT birthdate) FROM people;
```

In [32]:
import psycopg2 
conn = psycopg2.connect("dbname=test user=test")
cur = conn.cursor()
cur.execute(
    """
    CREATE TABLE postgis.tabla_psycopg (id serial PRIMARY KEY, num integer, data varchar);
    INSERT INTO postgis.tabla_psycopg VALUES (1, 2, 'mi_primera_tabla');
    """
)
conn.commit()
cur.close()
conn.close()